```{contents}
```
## **Environment Configuration in LangGraph**

Environment configuration in LangGraph defines **how a graph executes in different runtime contexts** — development, testing, staging, and production — controlling models, memory, persistence, security, observability, and system behavior without changing application code.

---

### **1. Why Environment Configuration Matters**

LLM workflows are **stateful, distributed, and expensive**.
Incorrect configuration leads to:

* Non-reproducible behavior
* Data leakage
* Uncontrolled cost
* System instability

LangGraph separates **graph logic** from **runtime configuration** to enable safe deployment.

---

### **2. Configuration Layers**

| Layer                 | Responsibility           |
| --------------------- | ------------------------ |
| Graph Definition      | Nodes, edges, state      |
| Runtime Config        | Execution behavior       |
| Infrastructure Config | Storage, memory, network |
| Security Config       | Secrets, access          |
| Observability Config  | Logs, metrics, traces    |

---

### **3. Core Environment Components**

| Component        | Purpose                         |
| ---------------- | ------------------------------- |
| LLM Provider     | Model selection and credentials |
| Memory Backend   | Redis, Postgres, Vector DB      |
| Checkpoint Store | Durable state recovery          |
| Thread Store     | Session & conversation tracking |
| Tracing Backend  | LangSmith / OpenTelemetry       |
| Execution Limits | Timeouts, recursion limits      |
| Cost Controls    | Token budgets, quotas           |
| Security         | Keys, secrets, RBAC             |

---

### **4. Configuration via Environment Variables**

LangGraph uses **environment variables** as its primary configuration channel.

```bash
# LLM
export OPENAI_API_KEY=...
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=...

# Execution
export LANGGRAPH_RECURSION_LIMIT=25
export LANGGRAPH_TIMEOUT=60

# Persistence
export LANGGRAPH_CHECKPOINTS=redis
export REDIS_URL=redis://localhost:6379

# Observability
export LANGSMITH_PROJECT=prod-agent
```

---

### **5. Runtime Configuration Object**

```python
config = {
    "configurable": {
        "thread_id": "user-123",
        "model": "gpt-4.1-mini",
        "temperature": 0.2
    },
    "recursion_limit": 20,
    "callbacks": [tracer]
}
```

Used at invocation time:

```python
graph.invoke(input, config=config)
```

---

### **6. Multi-Environment Strategy**

| Environment | Characteristics                    |
| ----------- | ---------------------------------- |
| Development | Fast models, local memory          |
| Testing     | Mock tools, deterministic models   |
| Staging     | Production mirror                  |
| Production  | Full persistence, auditing, quotas |

Example:

```bash
export APP_ENV=production
```

```python
if os.getenv("APP_ENV") == "production":
    model = "gpt-4.1"
else:
    model = "gpt-3.5-turbo"
```

---

### **7. Production Configuration Pattern**

```text
.env.dev
.env.test
.env.staging
.env.prod
```

Each file defines:

* Model
* Memory backend
* Logging level
* Token budgets
* Retry policies
* Rate limits

---

### **8. Configuration for Persistence & Recovery**

```python
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver("graph.db")
graph = builder.compile(checkpointer=checkpointer)
```

Configured by environment:

```bash
export LANGGRAPH_DB_PATH=/data/graph.db
```

---

### **9. Configuration for Safety & Control**

| Setting         | Purpose                |
| --------------- | ---------------------- |
| recursion_limit | Prevent infinite loops |
| timeout         | Stop runaway execution |
| max_tokens      | Cost control           |
| retry_policy    | Reliability            |
| human_gate      | Manual approval        |

---

### **10. Common Variants**

| Variant    | Use Case                          |
| ---------- | --------------------------------- |
| Local Dev  | SQLite + cheap model              |
| Cloud SaaS | Redis + Postgres + premium models |
| Enterprise | Encrypted stores + audit logging  |
| Edge       | In-memory + local models          |

---

### **11. Conceptual Model**

> **Graph Logic is static.
> Environment Configuration makes it production-ready.**

LangGraph uses configuration to transform a **research workflow** into a **reliable distributed system**.

### Demonstration

In [3]:
# ============================
# LangGraph Environment Configuration — One-Cell Demonstration
# ============================

import os
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver

# ----------- 1. Simulated Environment Setup -----------

os.environ["APP_ENV"] = "development"
os.environ["LANGGRAPH_DB_PATH"] = "graph_dev.db"
os.environ["OPENAI_API_KEY"] = "your_api_key_here"

# ----------- 2. State Definition -----------

class State(TypedDict):
    input: str
    output: str

# ----------- 3. Environment-Aware Node -----------

def llm_node(state: State):
    env = os.getenv("APP_ENV")

    if env == "production":
        model = "gpt-4.1"
        style = "precise"
    else:
        model = "gpt-3.5-turbo"
        style = "fast"

    result = f"[{env.upper()} | {model} | {style}] → {state['input']}"
    return {"output": result}

# ----------- 4. Build Graph -----------

builder = StateGraph(State)
builder.add_node("llm", llm_node)
builder.set_entry_point("llm")
builder.add_edge("llm", END)

# ----------- 5. Configure Persistence from Environment -----------

import sqlite3
db_path = os.getenv("LANGGRAPH_DB_PATH")
checkpointer = SqliteSaver(sqlite3.connect(db_path,check_same_thread=False))

graph = builder.compile(checkpointer=checkpointer)

# ----------- 6. Runtime Configuration -----------

config = {
    "configurable": {"thread_id": "user-42"},
    "recursion_limit": 5
}

# ----------- 7. Execute -----------

result = graph.invoke({"input": "Hello LangGraph"}, config=config)

# ----------- 8. Output -----------

print("Output:", result["output"])
print("Checkpoint DB:", db_path)


Output: [DEVELOPMENT | gpt-3.5-turbo | fast] → Hello LangGraph
Checkpoint DB: graph_dev.db
